# Lecture 04: Subqueries, Aggregation

In [ ]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

Today's database is a modified version of the Oakland police stops database as made public by the Stanford Open Policing Project: https://openpolicing.stanford.edu/data/ Read more about the database [here](https://openpolicing.stanford.edu/findings/).

If you are looking for how we adjusted the data, contact us and we can share our source notebook.

In [ ]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS stops_lecture'
!psql -h localhost -c 'CREATE DATABASE stops_lecture' 
!psql -h localhost -d stops_lecture -f data/stops_lecture.sql

## `jupysql`

We are going to be using the `jupysql` library to connect our notebook to a PostgreSQL database server on your jupyterhub account. The next cell should do the trick; you should not see any error messages after it completes.

In [ ]:
%load_ext sql

In [ ]:
%sql --close postgresql://127.0.0.1:5432/stops_lecture

In [ ]:
%sql postgresql://127.0.0.1:5432/stops_lecture

In [ ]:
%%sql
SELECT *
FROM Zips
LIMIT 10;

With cell magic (`%%sql`), we can also assign the result to a variable. Try adding `result <<` to the top line of the below cell, i.e., replace the first line with

``%%sql result <<``

In [ ]:
%%sql
SELECT *
FROM Stops
ORDER BY RANDOM()
LIMIT 10;

In [ ]:
result

## Exercise \#1

How do we find the stops that happened in the same location as the stop with ID 123?


In [ ]:
%%sql
SELECT *
FROM Stops
WHERE id = 123;

In [ ]:
%%sql
SELECT S1.id, S1.race,
       S1.location
FROM Stops as S1,   -- the "other stops"
     Stops as S2    -- the "stop that refers to ID 123"
WHERE S1.location = S2.location -- the "other stops" must match the ID 123 location
     AND S2.id = 123;

<br/>

Approach \#2 with CTE:

In [ ]:
%%sql
WITH Location123 AS (
    SELECT location
    FROM Stops
    WHERE id = 123
)
SELECT S.id, S.race,
       S.location
FROM Stops as S,
    Location123
WHERE S.location = Location123.location;

In [ ]:
%%sql
WITH Location123 AS (
    SELECT location
    FROM Stops
    WHERE id = 123
) SELECT * FROM Location123;

<br/>

Approach \#3 with scalar subquery:

In [ ]:
%%sql
SELECT S1.id, S1.race, S1.location
FROM Stops S1
WHERE S1.location = (
         SELECT S2.location
         FROM Stops
         S2 WHERE S2.id = 123);

## `EXISTS` Subquery (and `NOT EXISTS`)

In [ ]:
%%sql
SELECT DISTINCT Stops.location FROM Stops
WHERE NOT EXISTS (
  SELECT * FROM Zips
  WHERE Zips.location = 
          Stops.location);

# Exercise \#2

In [ ]:
%%sql
WITH StopZips AS (
    SELECT * FROM Stops NATURAL JOIN Zips
)
SELECT *
FROM StopZips SZ1
WHERE NOT EXISTS (
    -- prove that there is not another id
    -- for the same zipcode
    -- --> i.e., there is an id for the same zipcode
    SELECT *
    FROM StopZips SZ2
    WHERE SZ1.zipcode = SZ2.zipcode -- same zipcode as the table we're looking at
          AND SZ1.id != SZ2.id
);


Note -- there are no stops that re the only one in their zipcode:

In [ ]:
%%sql
SELECT Z1.zipcode, COUNT(*) as count
FROM Stops S1
  NATURAL JOIN Zips Z1
GROUP BY Z1.zipcode
ORDER BY count ASC;

# Exercise \#3

In [ ]:
%%sql
SELECT S1.id, S1.age
FROM Stops S1 
WHERE S1.age >= 
	MAX(S1.age) -- yes, we need to compare to a single value!

In [ ]:
%%sql
SELECT S1.id, S1.age
FROM Stops S1 
WHERE S1.age == 
	(SELECT MAX(S2.age)
     FROM Stops S2);

In [ ]:
%%sql
SELECT location,
       AVG(age) AS avgage,
       MIN(age) AS minage
FROM Stops
GROUP BY location;

# Exercise \#4

In [ ]:
%%sql
SELECT race, 
AVG (CASE WHEN location = 'West Oakland'
     THEN age
     ELSE NULL
     END
    ) AS west_oakland_avg,
AVG (CASE WHEN location = 'Rockridge'
     THEN age
     ELSE NULL
     END
    ) AS rockridge_Avg 
FROM Stops
GROUP BY race;